In [22]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn

#import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.metrics import r2_score

from itertools import product

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 

# Import Data
We are using this data from Kaggle which is very similar to the merchants' data from www.spaceship.com.sg 

https://www.kaggle.com/jagangupta/time-series-basics-exploring-traditional-ts/data

In [23]:
data_path = 'data/'
sales = pd.read_csv(data_path + 'sales_train.csv.gz')
item_cats = pd.read_csv(data_path + 'item_categories.csv')
items = pd.read_csv(data_path + 'items.csv')
shops = pd.read_csv(data_path + 'shops.csv')

#Use subset for simplicity
sales = sales[sales['shop_id'].isin([26, 27, 28])]

# Feature Engineering
We want to predict the target(items sold) in next month. 
The key idea is that we can use previous months items sales data to predict next month sales.
We view the data as monthly blocks based on 'date_block_num'. 
Initutively, items sales can be attribute to the months(seasonality) or the ability of the individual shops.
In each monthly block, we want to explore this relationship of the item-month and item-shop.

- Split the data into their monthly block number 
- For each month, we want to aggregrate total items sold per shop_id-item_id pairs

In [24]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

print all_data.head()

   shop_id  item_id  date_block_num  target  target_shop  target_item
0       28     7738               0     4.0       7057.0         11.0
1       28     7737               0    10.0       7057.0         16.0
2       28     7770               0     6.0       7057.0         10.0
3       28     7664               0     1.0       7057.0          1.0
4       28     7814               0     2.0       7057.0          6.0


# Target Encoding
Use target encoding of lagging sales data so as to encode seasonality for the prediction. 

In [25]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in shift_range:
    train_shift = all_data[index_cols + cols_to_rename].copy()
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift
# Remove the 1st 12 months because the are no previous 12 months lagging data
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

print all_data.columns

Index([u'shop_id', u'item_id', u'date_block_num', u'target', u'target_shop',
       u'target_item', u'target_lag_1', u'target_item_lag_1',
       u'target_shop_lag_1', u'target_lag_2', u'target_item_lag_2',
       u'target_shop_lag_2', u'target_lag_3', u'target_item_lag_3',
       u'target_shop_lag_3', u'target_lag_4', u'target_item_lag_4',
       u'target_shop_lag_4', u'target_lag_5', u'target_item_lag_5',
       u'target_shop_lag_5', u'target_lag_12', u'target_item_lag_12',
       u'target_shop_lag_12'],
      dtype='object')


# Train/test split
We will treat last month data as the test set.

In [26]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

Test `date_block_num` is 33


# First level models 
We will implement a stacking scheme. We have a time component here, so we will use KFold scheme in time series. We always use first level models to build two datasets: test meta-features and 2-nd level train meta-features. 
We first get test meta-features here. 

In [27]:
#linear regression 
model_lr = LinearRegression()
model_lr.fit(X_train.values, y_train)
pred_lr = model_lr.predict(X_test.values)
print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))

#LightGBM
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }
model_lgb = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model_lgb.predict(X_test)
print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))

#RandomForest
model_forest = RandomForestRegressor(max_depth=3, random_state=42)
model_forest.fit(X_train, y_train)
pred_forest = model_forest.predict(X_test)
print('Test R-squared for RandomForest is %f' % r2_score(y_test, pred_forest))

#concatenate test predictions to get test meta-features
X_test_level2 = np.c_[pred_lr, pred_lgb,pred_forest] 
print X_test_level2.shape
#print np.corrcoef([pred_lr, pred_lgb,pred_forest])

Test R-squared for linreg is 0.743144
Test R-squared for LightGBM is 0.707822
Test R-squared for RandomForest is 0.667004
(3354, 3)


### Train meta-features
#### KFold scheme in time series
We cannot do normal KFold in time series because we need to be careful not to be using future data to validate past data.

Split the train data into chunks of duration T. Select first M chunks.
Fit N diverse models on those M chunks and predict for the chunk M+1. Then fit those models on first M+1 chunks and predict for chunk M+2 and so on. After that, use all train data to fit models and get predictions for test. Now we will have meta-features for the chunks starting from number M+1 as well as meta-features for the test.

We have data for month 12 to 32(first 12 months was removed earlier). We use M=15. For the first fold, we will used month 12 to 26 in order to predict for month 27. The same logic apply for months 28, 29, 30, 31, 32

In [28]:
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

pd_stack = pd.DataFrame(index=all_data.index)
dates_train_level2.shape

(34404,)

In [29]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 3])

# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

#Building metafeatures which are the predicted output from different models
last_index = 0
for cur_block_num in [27, 28, 29, 30, 31, 32]: 
        
    #Split `X_train` into parts
    X_train_level1 = all_data.loc[dates <  cur_block_num].drop(to_drop_cols, axis=1)
    X_valid_level1 =  all_data.loc[dates == cur_block_num].drop(to_drop_cols, axis=1)

    y_train_level1 = all_data.loc[dates <  cur_block_num, 'target'].values
    y_valid_level1 =  all_data.loc[dates == cur_block_num, 'target'].values

    start_index = last_index
    last_index += y_valid_level1.shape[0]
    print "Processing block:",cur_block_num, " start_index:", start_index, " last_index:", last_index

    #Fit linear regression
    model_lr.fit(X_train_level1.values, y_train_level1)
    pred_lr = model_lr.predict(X_valid_level1.values)
    X_train_level2[start_index:last_index,0] = pred_lr
 
    #Fit lightGBM
    model_lbg = lgb.train(lgb_params, lgb.Dataset(X_train_level1, label=y_train_level1), 100)
    pred_lgb = model_lbg.predict(X_valid_level1)
    X_train_level2[start_index:last_index,1] = pred_lgb
    
    #Fit Random Forest
    model_forest.fit(X_train_level1.values, y_train_level1)
    pred_forest = model_forest.predict(X_valid_level1.values)
    X_train_level2[start_index:last_index,2] = pred_forest
    
print X_train_level2.mean(axis=0) # [1.50291139  1.36876467 1.32128003]

Processing block: 27  start_index: 0  last_index: 6438
Processing block: 28  start_index: 6438  last_index: 13242
Processing block: 29  start_index: 13242  last_index: 19935
Processing block: 30  start_index: 19935  last_index: 26409
Processing block: 31  start_index: 26409  last_index: 30027
Processing block: 32  start_index: 30027  last_index: 34404
[ 1.50291139  1.36876467  1.32128003]


# Stacking
Fit a linear regression model with the meta-features. Use this new level-2 meta model to predict test data

In [30]:
#Fit model
model_level2 = LinearRegression()
model_level2.fit(X_train_level2, y_train_level2)

#Train result
train_preds = model_level2.predict(X_train_level2) 
r2_train_stacking = r2_score(y_train_level2, train_preds)
print('Train R-squared for stacking is %f' % r2_train_stacking)

#test result
test_preds = model_level2.predict(X_test_level2)
r2_test_stacking = r2_score(y_test, test_preds)
print('Test R-squared for stacking is %f' % r2_test_stacking)

Train R-squared for stacking is 0.641681
Test R-squared for stacking is 0.759779


We achieve a score of 0.75 for test data. This can be further improved by stacking different models and exploring more external data which affect time series. One good example is public holidays which can affect sales of certain items a lot. 